## 0. Notebook Parameters

---

### Notebook Settings

In [ ]:
"""Google Colab settings"""
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [1]:
"""Jupyter settings"""
# Enable autoreload
%load_ext autoreload
%autoreload 2

# Pylint parameters
%config Completer.use_jedi = False

# Measure Runtime
# !pip install ipython-autotime
%load_ext autotime

time: 514 µs (started: 2021-03-01 00:38:39 +01:00)


### Imported Packages

In [2]:
"""Packages for manipulation of vectors, arrays, dataframes"""
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None) # Change display settings of class

"""Packages for cleaning dataset"""
import json
import string
import unicodedata

"""Packages for data visualization"""
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn

time: 872 ms (started: 2021-03-01 00:38:42 +01:00)


## 1. Preprocessing

---

### Clean OCR Dataset

#### Import the Data

In [5]:
# Load the dataset
file = '../raw_data/ocr_fr_text_only.jsonl'

def load_json_data(file):
    """Load the dataset"""
    data = []
    with open(file) as f:
        for line in f:
            data.append(json.loads(line))
    return pd.DataFrame(data)

ocr_df_base = load_json_data(file)

time: 6.04 s (started: 2021-02-28 23:38:48 +01:00)


In [6]:
# Deep copy of the dataframe to avoid to reload it
ocr_df = ocr_df_base.copy()

time: 27.4 ms (started: 2021-02-28 23:39:39 +01:00)


In [8]:
# Brief look at the dataset
print(f"Shape of the dataset: {ocr_df.shape}")
print("head of ocr_df:")
display(ocr_df.head())
print("Content of the first element:")
display(ocr_df.fr_text[0])

Shape of the dataset: (795609, 2)
head of ocr_df:


,source,fr_text
0,/319/966/047/6748/1.json,"NE\nELEVE\nABATTU\nEN BRETAGNE\nLES ÉLEVEURS\nDE BRETAGNE\nDécoupe de\nPodlet noir\nferfier\nlabe\nCaractéristiques certifiées: Fermier-élevé en plein air. Durée d'élevage 81 jours minimum.\n75% de céréales.\nAlimenté avec 100% de végétaux, minéraux et vitamines do mation : voir étiquette poids/prix\nA conserver entre O°C et +4°C -A consommer cuit à caeur-Date limite de co 35650 Le Rheu\nCertifié par: CERTIS-Immeuble Le Millepertuis- Les Landes d'Ap\nProduit frais\nClasse A\nPour toute réclamation, s'adresser à\nFERMIERS D'ARGOAT: BP77-22440 PLOUFRAGAN\nDécoupé et conditionné par: LDC Bretagne BP 256-2280 QUINTIN\nHomologation\nN° LA/02/75\n099 002\nCE\n2\nAbattoir agréé n'FR 22.099.002 CE\n2FLT.PLT FER.LR SAT\nEL.BRET.PF 2\nORIGINE France\nyo\nVOLAISE\nPrixtrg\nPoids net\nPrix a payer\n0,240kg\nLot\n005809 19 45\nR consommer jusqu au\nR conserver entre O C et +4 C\n10/04/18\nExpedie Le 30/03/18\nLOC BRETAGNE 22 LANFAINS\n22.039.002\ne01\n172225-0/196A\n3 19966014\nAU TRIT ASTE\nwww.CONSIGNESDETRFR\n"
1,/319/966/021/9208/1.json,"..\n1cg% LOCAL\nKer\nchAnt\nPOULET\nDecoupes de\nConditionné par D\nBretagne\nOrigine FRANCE. Produit Frais, Classe A\nconsommer cuit à coeur\nNE ELEVÉ PREPARE\n100% LOCAL\nDANS NOTRE REGION\nFILETS POOLET\n720GR\nVOLAILLE\nORIGINE France\nPrixtkg\nPoids net\nPrix a pave\n0,720kg\nO058067878\nA consommer jusqu au\n04/10/18\nCot\nA conserver entre O C et 4C\nExpedie Le\nLOc BRETAGNE 22 LANFAINS\n22.099.002\nCE\n250498-0/181\n01\n3 11199660121\nCONSGNE POUANT REOCLMENT\nwww.cONSIGNT\n"
2,/319/966/021/9208/2.json,FILETS POULET\n720GR\nORIGINE France\nPrixtkg\nVOLAILLE\nFRANÇAISE\nPoids net\nPrix a payer\n0720kg\nLot\n0058067878\nA consommer jusqu au\n04/10/18\nA conserver entre O C et 4 C\nExpedie Le\nLOC BRETAGNE 22 LANFAINS\nFR\nCE\nBARQUETTE ET\n22.099.002\n250498-0/18 1C\ne01\nPENSEZ\nAU TRI!\nCONSIGNE POUVANTVARIER LOCALEMENT\nFILM PLASTIQUE\nA JETER\n3 |199 66012 19208\nwww.CONSIGNESDETRI.FR\n
3,/319/966/021/9192/1.json,"Ker\nchant\n100% LOCAL\nDecoupes de\nPOULET\nConditionné par LDC Bretagne\nLanfains (22) 18\nrais Classe A\nOrigine FRANCE. Prodtre 0 et +4""C\nOFFRE\nSPECIALE\nNE ELEVE PREPARE\n100% LOCAL\nDANS NOTRE REGION\n1CUIS.DEJ. PLT SAT\nKERCHANT FX 1\nORIGINE France\nVOLAILLE\nFRANÇAISE\nPrix/kg\nPaids net\nPrin apayer\n1,000kg\nto\n005806444\nA consommer jusqu au\n27/09/18\nR conserver entre D C et 4 C\nExpedie Le\nLOC BRETAGNE 22 LANFAINS\n2.09.02\n256-106-0/1818\nAUTRIT\nPENS\n3 199660112 19192\n"
4,/319/966/021/9192/2.json,"1CUIS.DEJ. PLT SAT\nKERCHANT FX 1\nORIGINE France\nVOLAILLE\nFRANCAISE\nPrix/kg\nPoids net\nPrix a payer\n1,000kg\n0058064444\nA consommer jusqu'au\nR conserver entre OC et 4 C\nExpedie Le\n27/09/18\nLOC BRETAGNE 22 LANFAINS\n22.099.002\nCE\n256 106-0/1818\ne03\n3 199 660112 191921\nPENSEZ\nAU TRI!\nBARQUETTE ET\nFILM PLASTIQUE\nAJETER\nCONSIGNE POUVANTVARIER LOCALEMENT\nwww.CONSIGNESDETRLFR\n"


Content of the first element:


"NE\nELEVE\nABATTU\nEN BRETAGNE\nLES ÉLEVEURS\nDE BRETAGNE\nDécoupe de\nPodlet noir\nferfier\nlabe\nCaractéristiques certifiées: Fermier-élevé en plein air. Durée d'élevage 81 jours minimum.\n75% de céréales.\nAlimenté avec 100% de végétaux, minéraux et vitamines do mation : voir étiquette poids/prix\nA conserver entre O°C et +4°C -A consommer cuit à caeur-Date limite de co 35650 Le Rheu\nCertifié par: CERTIS-Immeuble Le Millepertuis- Les Landes d'Ap\nProduit frais\nClasse A\nPour toute réclamation, s'adresser à\nFERMIERS D'ARGOAT: BP77-22440 PLOUFRAGAN\nDécoupé et conditionné par: LDC Bretagne BP 256-2280 QUINTIN\nHomologation\nN° LA/02/75\n099 002\nCE\n2\nAbattoir agréé n'FR 22.099.002 CE\n2FLT.PLT FER.LR SAT\nEL.BRET.PF 2\nORIGINE France\nyo\nVOLAISE\nPrixtrg\nPoids net\nPrix a payer\n0,240kg\nLot\n005809 19 45\nR consommer jusqu au\nR conserver entre O C et +4 C\n10/04/18\nExpedie Le 30/03/18\nLOC BRETAGNE 22 LANFAINS\n22.039.002\ne01\n172225-0/196A\n3 19966014\nAU TRIT ASTE\nwww.C

time: 14 ms (started: 2021-02-28 23:41:00 +01:00)


#### Format the Dataset

In [9]:
# Format the content of the column SOURCE
# CREATE BARCODE

def to_barcode(text):
    """Transform strings in barcodes"""
    return int(''.join(text.split('/')[0:-1]))


# Apply function to create barcodes
ocr_df['barcode'] = ocr_df.source.apply(lambda text: to_barcode(text))

time: 692 ms (started: 2021-02-28 23:41:08 +01:00)


In [12]:
# Format the content of the column TEXT
# CLEAN OCR TEXT

def lower_case(text):
    """Lower case"""
    return text.lower()

def replace_slashn(text):
    """Replace '/n' """
    return text.replace('\n',' ')

def remove_punc(text):
    """Remove punctuation"""
    for punctuation in string.punctuation:
        return text.replace(punctuation, '')

def remove_digit(text):
    """Remove digits"""
    return ''.join(word for word in text if not word.isdigit())

def remove_nonalpha(text):
    """Remove nonalpha characters"""
    return ''.join(word for word in text if word.isalpha() or word == ' ')

def remove_accents(text):
    """Remove accents"""
    return ''.join(
        c for c in unicodedata.normalize('NFKD', text) 
        if unicodedata.category(c) != 'Mn'
        )

def remove_specialchar(text):
    """Remove special characters"""
    return ''.join(word for word in text if word.isalpha() or word == ' ')

def text_formatting_workflow():
    """Workflow of text formatting"""
    ocr_df['clean_text'] = ocr_df.fr_text.apply(lambda text: lower_case(text))
    ocr_df['clean_text'] = ocr_df.clean_text.apply(lambda text: replace_slashn(text))
    ocr_df['clean_text'] = ocr_df.clean_text.apply(lambda text: remove_punc(text))
    ocr_df['clean_text'] = ocr_df.clean_text.apply(lambda text: remove_digit(text))
    # ocr_df['clean_text'] = ocr_df.clean_text.apply(lambda text: remove_nonalpha(text))
    ocr_df['clean_text'] = ocr_df.clean_text.apply(lambda text: remove_accents(text))
    ocr_df['clean_text'] = ocr_df.clean_text.apply(lambda text: remove_specialchar(text))
    
    return ocr_df


,source,fr_text,barcode,clean_text
0,/319/966/047/6748/1.json,"NE\nELEVE\nABATTU\nEN BRETAGNE\nLES ÉLEVEURS\nDE BRETAGNE\nDécoupe de\nPodlet noir\nferfier\nlabe\nCaractéristiques certifiées: Fermier-élevé en plein air. Durée d'élevage 81 jours minimum.\n75% de céréales.\nAlimenté avec 100% de végétaux, minéraux et vitamines do mation : voir étiquette poids/prix\nA conserver entre O°C et +4°C -A consommer cuit à caeur-Date limite de co 35650 Le Rheu\nCertifié par: CERTIS-Immeuble Le Millepertuis- Les Landes d'Ap\nProduit frais\nClasse A\nPour toute réclamation, s'adresser à\nFERMIERS D'ARGOAT: BP77-22440 PLOUFRAGAN\nDécoupé et conditionné par: LDC Bretagne BP 256-2280 QUINTIN\nHomologation\nN° LA/02/75\n099 002\nCE\n2\nAbattoir agréé n'FR 22.099.002 CE\n2FLT.PLT FER.LR SAT\nEL.BRET.PF 2\nORIGINE France\nyo\nVOLAISE\nPrixtrg\nPoids net\nPrix a payer\n0,240kg\nLot\n005809 19 45\nR consommer jusqu au\nR conserver entre O C et +4 C\n10/04/18\nExpedie Le 30/03/18\nLOC BRETAGNE 22 LANFAINS\n22.039.002\ne01\n172225-0/196A\n3 19966014\nAU TRIT ASTE\nwww.CONSIGNESDETRFR\n",3199660476748,ne eleve abattu en bretagne les eleveurs de bretagne decoupe de podlet noir ferfier labe caracteristiques certifiees fermiereleve en plein air duree delevage jours minimum de cereales alimente avec de vegetaux mineraux et vitamines do mation voir etiquette poidsprix a conserver entre oc et c a consommer cuit a caeurdate limite de co le rheu certifie par certisimmeuble le millepertuis les landes dap produit frais classe a pour toute reclamation sadresser a fermiers dargoat bp ploufragan decoupe et conditionne par ldc bretagne bp quintin homologation n la ce abattoir agree nfr ce fltplt ferlr sat elbretpf origine france yo volaise prixtrg poids net prix a payer kg lot r consommer jusqu au r conserver entre o c et c expedie le loc bretagne lanfains e a au trit aste wwwconsignesdetrfr
1,/319/966/021/9208/1.json,"..\n1cg% LOCAL\nKer\nchAnt\nPOULET\nDecoupes de\nConditionné par D\nBretagne\nOrigine FRANCE. Produit Frais, Classe A\nconsommer cuit à coeur\nNE ELEVÉ PREPARE\n100% LOCAL\nDANS NOTRE REGION\nFILETS POOLET\n720GR\nVOLAILLE\nORIGINE France\nPrixtkg\nPoids net\nPrix a pave\n0,720kg\nO058067878\nA consommer jusqu au\n04/10/18\nCot\nA conserver entre O C et 4C\nExpedie Le\nLOc BRETAGNE 22 LANFAINS\n22.099.002\nCE\n250498-0/181\n01\n3 11199660121\nCONSGNE POUANT REOCLMENT\nwww.cONSIGNT\n",3199660219208,cg local ker chant poulet decoupes de conditionne par d bretagne origine france produit frais classe a consommer cuit a coeur ne eleve prepare local dans notre region filets poolet gr volaille origine france prixtkg poids net prix a pave kg o a consommer jusqu au cot a conserver entre o c et c expedie le loc bretagne lanfains ce consgne pouant reoclment wwwconsignt
2,/319/966/021/9208/2.json,FILETS POULET\n720GR\nORIGINE France\nPrixtkg\nVOLAILLE\nFRANÇAISE\nPoids net\nPrix a payer\n0720kg\nLot\n0058067878\nA consommer jusqu au\n04/10/18\nA conserver entre O C et 4 C\nExpedie Le\nLOC BRETAGNE 22 LANFAINS\nFR\nCE\nBARQUETTE ET\n22.099.002\n250498-0/18 1C\ne01\nPENSEZ\nAU TRI!\nCONSIGNE POUVANTVARIER LOCALEMENT\nFILM PLASTIQUE\nA JETER\n3 |199 66012 19208\nwww.CONSIGNESDETRI.FR\n,3199660219208,filets poulet gr origine france prixtkg volaille francaise poids net prix a payer kg lot a consommer jusqu au a conserver entre o c et c expedie le loc bretagne lanfains fr ce barquette et c e pensez au tri consigne pouvantvarier localement film plastique a jeter wwwconsignesdetrifr
3,/319/966/021/9192/1.json,"Ker\nchant\n100% LOCAL\nDecoupes de\nPOULET\nConditionné par LDC Bretagne\nLanfains (22) 18\nrais Classe A\nOrigine FRANCE. Prodtre 0 et +4""C\nOFFRE\nSPECIALE\nNE ELEVE PREPARE\n100% LOCAL\nDANS NOTRE REGION\n1CUIS.DEJ. PLT SAT\nKERCHANT FX 1\nORIGINE France\nVOLAILLE\nFRANÇAISE\nPrix/kg\nPaids net\nPrin apayer\n1,000kg\nto\n005806444\nA consommer jusqu au\n27/09/18\nR conserver entre D C et 4 C\nExpedie Le\nLOC BRETAGNE 22 LANFAINS\n2.09.02\n256-106-0/1818\nAUTR

time: 1min 49s (started: 2021-02-28 23:43:20 +01:00)


In [13]:
# Apply workflow of text formatting
text_formatting_workflow()

,source,fr_text,barcode,clean_text
0,/319/966/047/6748/1.json,"NE\nELEVE\nABATTU\nEN BRETAGNE\nLES ÉLEVEURS\nDE BRETAGNE\nDécoupe de\nPodlet noir\nferfier\nlabe\nCaractéristiques certifiées: Fermier-élevé en plein air. Durée d'élevage 81 jours minimum.\n75% de céréales.\nAlimenté avec 100% de végétaux, minéraux et vitamines do mation : voir étiquette poids/prix\nA conserver entre O°C et +4°C -A consommer cuit à caeur-Date limite de co 35650 Le Rheu\nCertifié par: CERTIS-Immeuble Le Millepertuis- Les Landes d'Ap\nProduit frais\nClasse A\nPour toute réclamation, s'adresser à\nFERMIERS D'ARGOAT: BP77-22440 PLOUFRAGAN\nDécoupé et conditionné par: LDC Bretagne BP 256-2280 QUINTIN\nHomologation\nN° LA/02/75\n099 002\nCE\n2\nAbattoir agréé n'FR 22.099.002 CE\n2FLT.PLT FER.LR SAT\nEL.BRET.PF 2\nORIGINE France\nyo\nVOLAISE\nPrixtrg\nPoids net\nPrix a payer\n0,240kg\nLot\n005809 19 45\nR consommer jusqu au\nR conserver entre O C et +4 C\n10/04/18\nExpedie Le 30/03/18\nLOC BRETAGNE 22 LANFAINS\n22.039.002\ne01\n172225-0/196A\n3 19966014\nAU TRIT ASTE\nwww.CONSIGNESDETRFR\n",3199660476748,ne eleve abattu en bretagne les eleveurs de bretagne decoupe de podlet noir ferfier labe caracteristiques certifiees fermiereleve en plein air duree delevage jours minimum de cereales alimente avec de vegetaux mineraux et vitamines do mation voir etiquette poidsprix a conserver entre oc et c a consommer cuit a caeurdate limite de co le rheu certifie par certisimmeuble le millepertuis les landes dap produit frais classe a pour toute reclamation sadresser a fermiers dargoat bp ploufragan decoupe et conditionne par ldc bretagne bp quintin homologation n la ce abattoir agree nfr ce fltplt ferlr sat elbretpf origine france yo volaise prixtrg poids net prix a payer kg lot r consommer jusqu au r conserver entre o c et c expedie le loc bretagne lanfains e a au trit aste wwwconsignesdetrfr
1,/319/966/021/9208/1.json,"..\n1cg% LOCAL\nKer\nchAnt\nPOULET\nDecoupes de\nConditionné par D\nBretagne\nOrigine FRANCE. Produit Frais, Classe A\nconsommer cuit à coeur\nNE ELEVÉ PREPARE\n100% LOCAL\nDANS NOTRE REGION\nFILETS POOLET\n720GR\nVOLAILLE\nORIGINE France\nPrixtkg\nPoids net\nPrix a pave\n0,720kg\nO058067878\nA consommer jusqu au\n04/10/18\nCot\nA conserver entre O C et 4C\nExpedie Le\nLOc BRETAGNE 22 LANFAINS\n22.099.002\nCE\n250498-0/181\n01\n3 11199660121\nCONSGNE POUANT REOCLMENT\nwww.cONSIGNT\n",3199660219208,cg local ker chant poulet decoupes de conditionne par d bretagne origine france produit frais classe a consommer cuit a coeur ne eleve prepare local dans notre region filets poolet gr volaille origine france prixtkg poids net prix a pave kg o a consommer jusqu au cot a conserver entre o c et c expedie le loc bretagne lanfains ce consgne pouant reoclment wwwconsignt
2,/319/966/021/9208/2.json,FILETS POULET\n720GR\nORIGINE France\nPrixtkg\nVOLAILLE\nFRANÇAISE\nPoids net\nPrix a payer\n0720kg\nLot\n0058067878\nA consommer jusqu au\n04/10/18\nA conserver entre O C et 4 C\nExpedie Le\nLOC BRETAGNE 22 LANFAINS\nFR\nCE\nBARQUETTE ET\n22.099.002\n250498-0/18 1C\ne01\nPENSEZ\nAU TRI!\nCONSIGNE POUVANTVARIER LOCALEMENT\nFILM PLASTIQUE\nA JETER\n3 |199 66012 19208\nwww.CONSIGNESDETRI.FR\n,3199660219208,filets poulet gr origine france prixtkg volaille francaise poids net prix a payer kg lot a consommer jusqu au a conserver entre o c et c expedie le loc bretagne lanfains fr ce barquette et c e pensez au tri consigne pouvantvarier localement film plastique a jeter wwwconsignesdetrifr
3,/319/966/021/9192/1.json,"Ker\nchant\n100% LOCAL\nDecoupes de\nPOULET\nConditionné par LDC Bretagne\nLanfains (22) 18\nrais Classe A\nOrigine FRANCE. Prodtre 0 et +4""C\nOFFRE\nSPECIALE\nNE ELEVE PREPARE\n100% LOCAL\nDANS NOTRE REGION\n1CUIS.DEJ. PLT SAT\nKERCHANT FX 1\nORIGINE France\nVOLAILLE\nFRANÇAISE\nPrix/kg\nPaids net\nPrin apayer\n1,000kg\nto\n005806444\nA consommer jusqu au\n27/09/18\nR conserver entre D C et 4 C\nExpedie Le\nLOC BRETAGNE 22 LANFAINS\n2.09.02\n256-106-0/1818\nAUTR

time: 1min 54s (started: 2021-02-28 23:45:09 +01:00)


In [14]:
# Drop superfluous columns

# Rough variant (keeping the raw text as well)
ocr_rough_df = ocr_df[['barcode','clean_text', 'fr_text']].copy()
# Replace column names of the dataset
ocr_rough_df.rename(columns={'fr_text':'raw_text'}, inplace=True)

# Clean variant
ocr_clean_df = ocr_df[['barcode','clean_text']].copy()

time: 240 ms (started: 2021-02-28 23:47:04 +01:00)


In [15]:
ocr_rough_df.dtypes , ocr_clean_df.dtypes

(barcode        int64
 clean_text    object
 raw_text      object
 dtype: object,
 barcode        int64
 clean_text    object
 dtype: object)

time: 4.96 ms (started: 2021-02-28 23:47:04 +01:00)


In [16]:
"""Uncomment method if needed"""

# Save formatted dataset to csv
# ocr_clean_df.to_csv('/content/drive/My Drive/OFF_Drive/raw_data/ocr_clean.csv', index = False)
# ocr_rough_df.to_csv('/content/drive/My Drive/OFF_Drive/raw_data/ocr_clean_vs_rough.csv', index = False)

'Uncomment method if needed'

time: 4.68 ms (started: 2021-02-28 23:47:11 +01:00)


### Clean OFF Dataset

#### Import the Data

In [18]:
# Load the dataset
file = '../raw_data/en.openfoodfacts.org.products.csv'
off_df_base = pd.read_csv(file, sep='\t')

time: 8min 59s (started: 2021-02-28 23:48:06 +01:00)


In [19]:
# Deep copy of the dataframe to avoid to reload it
data_off = off_df_base.copy()

time: 7.76 s (started: 2021-02-28 23:57:06 +01:00)


In [20]:
# Brief look at the dataset
print(f"Shape of the dataset: {data_off.shape}")
print("head of data_off:")
display(data_off.head())

Shape of the dataset: (1634734, 184)
head of data_off:


,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,abbreviated_product_name,generic_name,...,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
0,0000000000017,http://world-en.openfoodfacts.org/product/0000000000017/vitoria-crackers,kiliweb,1529059080,2018-06-15T10:38:00Z,1561463718,2019-06-25T11:55:18Z,Vitória crackers,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000000031,http://world-en.openfoodfacts.org/product/0000000000031/cacao,isagoofy,1539464774,2018-10-13T21:06:14Z,1539464817,2018-10-13T21:06:57Z,Cacao,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000000000003327986,http://world-en.openfoodfacts.org/product/000000000003327986/filetes-de-pollo-empanado,kiliweb,1574175736,2019-11-19T15:02:16Z,1574175737,2019-11-19T15:02:17Z,Filetes de pollo empanado,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0000000000100,http://world-en.openfoodfacts.org/product/0000000000100/moutarde-au-mout-de-raisin-courte-paille,del51,1444572561,2015-10-11T14:09:21Z,1444659212,2015-10-12T14:13:32Z,moutarde au moût de raisin,NaN,NaN,...,NaN,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00000000001111111111,http://world-en.openfoodfacts.org/product/00000000001111111111/sfiudwx-watt,openfoodfacts-contributors,1560020173,2019-06-08T18:56:13Z,1560020173,2019-06-08T18:56:13Z,Sfiudwx,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


time: 2.09 s (started: 2021-02-28 23:57:14 +01:00)


#### Format the Dataset

In [21]:
# Drop columns
data_off_small = data_off[['code', 'pnns_groups_1', 'pnns_groups_2']].copy()

time: 137 ms (started: 2021-02-28 23:58:08 +01:00)


In [22]:
# Replace empty values with NaN
data_off_small['pnns_groups_1'] = data_off_small['pnns_groups_1'].replace('unknown',np.nan)
data_off_small['pnns_groups_2'] = data_off_small['pnns_groups_2'].replace('unknown',np.nan)

time: 131 ms (started: 2021-02-28 23:58:13 +01:00)


In [23]:
# Drop rows with NaN
off_df = data_off_small.dropna(subset=['pnns_groups_1', 'pnns_groups_2']).copy()

time: 266 ms (started: 2021-02-28 23:59:41 +01:00)


In [24]:
# Brief look at the dataset
print(f"Shape of the dataset: {off_df.shape}")
print("head of the dataset:")
display(off_df.head())

Shape of the dataset: (641233, 3)
head of the dataset:


,code,pnns_groups_1,pnns_groups_2
3,0000000000100,Fat and sauces,Dressings and sauces
14,0000000000949,Composite foods,One-dish meals
23,0000000001281,Sugary snacks,Biscuits and cakes
33,0000000001885,Fruits and vegetables,Fruits
35,0000000002103,Fish Meat Eggs,Meat


time: 13.6 ms (started: 2021-02-28 23:59:44 +01:00)


In [25]:
# Replace column names of the dataset
off_df.rename(columns={'code':'barcode'}, inplace=True)

time: 3.29 ms (started: 2021-02-28 23:59:49 +01:00)


In [26]:
# Verify columns of the dataset
off_df.dtypes

barcode          object
pnns_groups_1    object
pnns_groups_2    object
dtype: object

time: 7.83 ms (started: 2021-02-28 23:59:52 +01:00)


In [27]:
"""Uncomment method if needed"""

# Save formatted dataset to csv
# off_df.to_csv('/content/drive/My Drive/OFF_Drive/raw_data/off_categories_only.csv', index = False)

'Uncomment method if needed'

time: 3.15 ms (started: 2021-02-28 23:59:59 +01:00)


### Merge OCR & OFF Datasets

Columns of the final dataset (rough variant):
- `barcode`
- `clean_text`
- `raw_text`
- `pnns_group_1`
- `pnns_group_2`

In [28]:
# Merge the 2 dataframes
merged_df = ocr_rough_df.merge(off_df, how='left', on='barcode')

time: 779 ms (started: 2021-03-01 00:00:07 +01:00)


In [29]:
# Brief look at the dataset
print(f"Shape of the dataset: {merged_df.shape}")
print("head of the dataset:")
display(merged_df.head())

Shape of the dataset: (795609, 5)
head of the dataset:


,barcode,clean_text,raw_text,pnns_groups_1,pnns_groups_2
0,3199660476748,ne eleve abattu en bretagne les eleveurs de bretagne decoupe de podlet noir ferfier labe caracteristiques certifiees fermiereleve en plein air duree delevage jours minimum de cereales alimente avec de vegetaux mineraux et vitamines do mation voir etiquette poidsprix a conserver entre oc et c a consommer cuit a caeurdate limite de co le rheu certifie par certisimmeuble le millepertuis les landes dap produit frais classe a pour toute reclamation sadresser a fermiers dargoat bp ploufragan decoupe et conditionne par ldc bretagne bp quintin homologation n la ce abattoir agree nfr ce fltplt ferlr sat elbretpf origine france yo volaise prixtrg poids net prix a payer kg lot r consommer jusqu au r conserver entre o c et c expedie le loc bretagne lanfains e a au trit aste wwwconsignesdetrfr,"NE\nELEVE\nABATTU\nEN BRETAGNE\nLES ÉLEVEURS\nDE BRETAGNE\nDécoupe de\nPodlet noir\nferfier\nlabe\nCaractéristiques certifiées: Fermier-élevé en plein air. Durée d'élevage 81 jours minimum.\n75% de céréales.\nAlimenté avec 100% de végétaux, minéraux et vitamines do mation : voir étiquette poids/prix\nA conserver entre O°C et +4°C -A consommer cuit à caeur-Date limite de co 35650 Le Rheu\nCertifié par: CERTIS-Immeuble Le Millepertuis- Les Landes d'Ap\nProduit frais\nClasse A\nPour toute réclamation, s'adresser à\nFERMIERS D'ARGOAT: BP77-22440 PLOUFRAGAN\nDécoupé et conditionné par: LDC Bretagne BP 256-2280 QUINTIN\nHomologation\nN° LA/02/75\n099 002\nCE\n2\nAbattoir agréé n'FR 22.099.002 CE\n2FLT.PLT FER.LR SAT\nEL.BRET.PF 2\nORIGINE France\nyo\nVOLAISE\nPrixtrg\nPoids net\nPrix a payer\n0,240kg\nLot\n005809 19 45\nR consommer jusqu au\nR conserver entre O C et +4 C\n10/04/18\nExpedie Le 30/03/18\nLOC BRETAGNE 22 LANFAINS\n22.039.002\ne01\n172225-0/196A\n3 19966014\nAU TRIT ASTE\nwww.CONSIGNESDETRFR\n",Fish Meat Eggs,Meat
1,3199660219208,cg local ker chant poulet decoupes de conditionne par d bretagne origine france produit frais classe a consommer cuit a coeur ne eleve prepare local dans notre region filets poolet gr volaille origine france prixtkg poids net prix a pave kg o a consommer jusqu au cot a conserver entre o c et c expedie le loc bretagne lanfains ce consgne pouant reoclment wwwconsignt,"..\n1cg% LOCAL\nKer\nchAnt\nPOULET\nDecoupes de\nConditionné par D\nBretagne\nOrigine FRANCE. Produit Frais, Classe A\nconsommer cuit à coeur\nNE ELEVÉ PREPARE\n100% LOCAL\nDANS NOTRE REGION\nFILETS POOLET\n720GR\nVOLAILLE\nORIGINE France\nPrixtkg\nPoids net\nPrix a pave\n0,720kg\nO058067878\nA consommer jusqu au\n04/10/18\nCot\nA conserver entre O C et 4C\nExpedie Le\nLOc BRETAGNE 22 LANFAINS\n22.099.002\nCE\n250498-0/181\n01\n3 11199660121\nCONSGNE POUANT REOCLMENT\nwww.cONSIGNT\n",NaN,NaN
2,3199660219208,filets poulet gr origine france prixtkg volaille francaise poids net prix a payer kg lot a consommer jusqu au a conserver entre o c et c expedie le loc bretagne lanfains fr ce barquette et c e pensez au tri consigne pouvantvarier localement film plastique a jeter wwwconsignesdetrifr,FILETS POULET\n720GR\nORIGINE France\nPrixtkg\nVOLAILLE\nFRANÇAISE\nPoids net\nPrix a payer\n0720kg\nLot\n0058067878\nA consommer jusqu au\n04/10/18\nA conserver entre O C et 4 C\nExpedie Le\nLOC BRETAGNE 22 LANFAINS\nFR\nCE\nBARQUETTE ET\n22.099.002\n250498-0/18 1C\ne01\nPENSEZ\nAU TRI!\nCONSIGNE POUVANTVARIER LOCALEMENT\nFILM PLASTIQUE\nA JETER\n3 |199 66012 19208\nwww.CONSIGNESDETRI.FR\n,NaN,NaN
3,3199660219192,ker chant local decoupes de poulet conditionne par ldc bretagne lanfains rais classe a origine france prodtre et c offre speciale ne eleve prepare local dans notre region cuisdej plt sat kerchant fx origine france volaille francaise prixkg paids net prin apayer kg to a consommer jusqu au r conserver entre d c et c expedie le loc bretagne lanfains autrit pens,"Ker\nchant\n100% LOCAL\nDecoupes de\nPOULET\nConditionné par LDC Bretagne\nLanfains (22) 18\nrais Classe A\nOrigine FRANCE. Prodtre 0 e

time: 12.1 ms (started: 2021-03-01 00:00:26 +01:00)


In [30]:
# Drop rows with NaN from the DataFrame
labeled_df = merged_df.dropna(subset=['pnns_groups_1', 'pnns_groups_2'])

time: 291 ms (started: 2021-03-01 00:00:31 +01:00)


In [31]:
# Brief look at the dataset
print(f"Shape of the dataset: {labeled_df.shape}")
print("head of the dataset:")
display(labeled_df.head())

Shape of the dataset: (434896, 5)
head of the dataset:


,barcode,clean_text,raw_text,pnns_groups_1,pnns_groups_2
0,3199660476748,ne eleve abattu en bretagne les eleveurs de bretagne decoupe de podlet noir ferfier labe caracteristiques certifiees fermiereleve en plein air duree delevage jours minimum de cereales alimente avec de vegetaux mineraux et vitamines do mation voir etiquette poidsprix a conserver entre oc et c a consommer cuit a caeurdate limite de co le rheu certifie par certisimmeuble le millepertuis les landes dap produit frais classe a pour toute reclamation sadresser a fermiers dargoat bp ploufragan decoupe et conditionne par ldc bretagne bp quintin homologation n la ce abattoir agree nfr ce fltplt ferlr sat elbretpf origine france yo volaise prixtrg poids net prix a payer kg lot r consommer jusqu au r conserver entre o c et c expedie le loc bretagne lanfains e a au trit aste wwwconsignesdetrfr,"NE\nELEVE\nABATTU\nEN BRETAGNE\nLES ÉLEVEURS\nDE BRETAGNE\nDécoupe de\nPodlet noir\nferfier\nlabe\nCaractéristiques certifiées: Fermier-élevé en plein air. Durée d'élevage 81 jours minimum.\n75% de céréales.\nAlimenté avec 100% de végétaux, minéraux et vitamines do mation : voir étiquette poids/prix\nA conserver entre O°C et +4°C -A consommer cuit à caeur-Date limite de co 35650 Le Rheu\nCertifié par: CERTIS-Immeuble Le Millepertuis- Les Landes d'Ap\nProduit frais\nClasse A\nPour toute réclamation, s'adresser à\nFERMIERS D'ARGOAT: BP77-22440 PLOUFRAGAN\nDécoupé et conditionné par: LDC Bretagne BP 256-2280 QUINTIN\nHomologation\nN° LA/02/75\n099 002\nCE\n2\nAbattoir agréé n'FR 22.099.002 CE\n2FLT.PLT FER.LR SAT\nEL.BRET.PF 2\nORIGINE France\nyo\nVOLAISE\nPrixtrg\nPoids net\nPrix a payer\n0,240kg\nLot\n005809 19 45\nR consommer jusqu au\nR conserver entre O C et +4 C\n10/04/18\nExpedie Le 30/03/18\nLOC BRETAGNE 22 LANFAINS\n22.039.002\ne01\n172225-0/196A\n3 19966014\nAU TRIT ASTE\nwww.CONSIGNESDETRFR\n",Fish Meat Eggs,Meat
3,3199660219192,ker chant local decoupes de poulet conditionne par ldc bretagne lanfains rais classe a origine france prodtre et c offre speciale ne eleve prepare local dans notre region cuisdej plt sat kerchant fx origine france volaille francaise prixkg paids net prin apayer kg to a consommer jusqu au r conserver entre d c et c expedie le loc bretagne lanfains autrit pens,"Ker\nchant\n100% LOCAL\nDecoupes de\nPOULET\nConditionné par LDC Bretagne\nLanfains (22) 18\nrais Classe A\nOrigine FRANCE. Prodtre 0 et +4""C\nOFFRE\nSPECIALE\nNE ELEVE PREPARE\n100% LOCAL\nDANS NOTRE REGION\n1CUIS.DEJ. PLT SAT\nKERCHANT FX 1\nORIGINE France\nVOLAILLE\nFRANÇAISE\nPrix/kg\nPaids net\nPrin apayer\n1,000kg\nto\n005806444\nA consommer jusqu au\n27/09/18\nR conserver entre D C et 4 C\nExpedie Le\nLOC BRETAGNE 22 LANFAINS\n2.09.02\n256-106-0/1818\nAUTRIT\nPENS\n3 199660112 19192\n",Fish Meat Eggs,Meat
4,3199660219192,cuisdej plt sat kerchant fx origine france volaille francaise prixkg poids net prix a payer kg a consommer jusquau r conserver entre oc et c expedie le loc bretagne lanfains ce e pensez au tri barquette et film plastique ajeter consigne pouvantvarier localement wwwconsignesdetrlfr,"1CUIS.DEJ. PLT SAT\nKERCHANT FX 1\nORIGINE France\nVOLAILLE\nFRANCAISE\nPrix/kg\nPoids net\nPrix a payer\n1,000kg\n0058064444\nA consommer jusqu'au\nR conserver entre OC et 4 C\nExpedie Le\n27/09/18\nLOC BRETAGNE 22 LANFAINS\n22.099.002\nCE\n256 106-0/1818\ne03\n3 199 660112 191921\nPENSEZ\nAU TRI!\nBARQUETTE ET\nFILM PLASTIQUE\nAJETER\nCONSIGNE POUVANTVARIER LOCALEMENT\nwww.CONSIGNESDETRLFR\n",Fish Meat Eggs,Meat
8,3199660747848,ker chant logal decoupes de poulet cooduogne pnm lin ne eleve prepare local aiguillette poulet gr dans not origine france prixiky tui et prix a pavar kg region lot consommer jusquau r conserver entre cet c expedie le loc bretrgne lrnfrins a pense au tri,"Ker\nChant\n100% LOGAL\nDécoupes de\nPOULET\nCooduogne pNM LIN\nNE\nELEVE\nPREPARE\n100% LOCAL\nAIGUILLETTE POULET\n500GR\nDANS NOT\nORIGINE France\nPrixiky\ntui et\nPrix a pavar\n0,500kg\nREGION\n0059308701\nLot

time: 16.1 ms (started: 2021-03-01 00:00:53 +01:00)


In [32]:
# Check the number of barcodes
print(f"Count of unique barcodes: {labeled_df.barcode.nunique()}")

Count of unique barcodes: 127888
time: 66.6 ms (started: 2021-03-01 00:00:57 +01:00)


In [33]:
"""Uncomment method if needed"""
# Save sample dataset to csv
# labeled_df.to_csv('../raw_data/ocr_labeled_rough.csv', index = False)

'Uncomment method if needed'

time: 4.03 ms (started: 2021-03-01 00:01:00 +01:00)


In [34]:
# Create a subset dataframe to test the code without waiting time
labeled_df_1K = labeled_df[:1000].copy()

"""Uncomment method if needed"""
# Save sample dataset to csv
# labeled_df_1K.to_csv('../raw_data/ocr_labeled_1K.csv', index = False)

'Uncomment method if needed'

time: 4.78 ms (started: 2021-03-01 00:01:04 +01:00)
